In [1]:
import numpy as np
import pandas as pd
import csv 
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import K2Score
from itertools import combinations
from pgmpy.estimators import BDeuScore
import sys
import time

In [2]:
# Hyperparameters
percentage_move = 0.3

In [3]:
def preprocess_data(ticker):
    data = pd.read_csv(f'ohlc_data_{ticker}.csv')  # Read CSV for current ticker
    
    # Similar preprocessing steps as you did for AAPL
    columns_to_remove = ['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Date']
    data = data.drop(columns=columns_to_remove, axis=1)
    data = data.rename_axis('day')
    data['day-1_close'] = data['Close'].shift(-1)
    data = data.iloc[:data.shape[0] - 1]
    data = data.dropna()

    # Calculate median of the 'Close' column (excluding first two and last two values)
    median_close = data['Close'][2:-2].median()
    # Calculate bins based on the median_close
    bins = [float('-inf'), -median_close* 0.003, median_close* 0.003, float('inf')]
    
    data[ticker] = pd.cut(
        data['day-1_close'] - data['Close'],
        bins=bins,
        labels=[-1, 0, 1]
    )
    data.rename(columns={ticker: f'{ticker}'}, inplace=True)
    columns_to_remove = ['Close', 'day-1_close']
    data = data.drop(columns=columns_to_remove, axis=1)
    
    results[ticker] = data
    all_data_frames.append(results[ticker])

In [ ]:
all_data_frames = []
ticker_symbols = ['AAPL', 'GOOGL', 'MSFT','ADBE', 'AMZN', 'BA','BRK-B', 'CSCO','HD', 'HDFCBANK.NS',
                  'IBM','INFY.NS', 'INTC', 'JNJ','JPM', 'KO', 'MA','MCD', 'META','NFLX', 
                  'NVDA', 'PEP','PG','PYPL', 'RELIANCE.NS', 'T','TCS.NS', 'TSLA', 'V','WMT']  # List of ticker symbols

results = {}  # Dictionary to store results for each ticker

for ticker in ticker_symbols:
    preprocess_data(ticker)
    
final = pd.concat(all_data_frames, axis=1)
print(final.head())

In [ ]:
data = [('GOOGL', 'AAPL'), ('HD', 'AAPL')]
sorted_data = sorted(data, key=lambda x: x[0])
model = BayesianNetwork(sorted_data)

In [ ]:
from pgmpy.estimators import BayesianEstimator
cpd1 = BayesianEstimator(model, final).estimate_cpd('AAPL', prior_type="K2")
print(cpd1)

+---------+----------+
| GOOGL   | GOOGL(0) |
+---------+----------+
| HD      | HD(0)    |
+---------+----------+
| AAPL(0) | 1.0      |
+---------+----------+


In [ ]:
def k2_score(data,final,ticker):
    all_combinations = []
    for r in range(1, len(data) + 1):
        combos = combinations(data, r)
        for combo in combos:
            # Convert the tuple to a list and append it
            combo_list = [item for sublist in combo for item in sublist]
            all_combinations.append(combo_list)
    null = []
    all_combinations.append(null)
    max_int = sys.maxsize
    k2score = K2Score(final)
    result = -max_int
    total_score = 0  # Use a different variable name than 'sum'
    combi = None
    
    for combo in all_combinations:
        k2_score = k2score.local_score(ticker, combo)
        if k2_score > result:
            result = k2_score
            combi = combo
    
    if combi is not None:
        print("Best combination according to K2 Score :", combi)
        print("Best Score according to K2 Score : " , result)
    else:
        print("No best combination found.")
        
    return combi, result


def bdeu_score(data,final,ticker):
    all_combinations = []
    for r in range(1, len(data) + 1):
        combos = combinations(data, r)
        for combo in combos:
            # Convert the tuple to a list and append it
            combo_list = [item for sublist in combo for item in sublist]
            all_combinations.append(combo_list)
    null = []
    all_combinations.append(null)
    max_int = sys.maxsize
    # Create an instance of BDeuScore with your model (assuming 'final' is your Bayesian network model)
    bdeu_score = BDeuScore(final)
    result = -max_int
    #total_score = 0  # Use a different variable name than 'sum'
    combi = None
    
    for combo in all_combinations:
        # Calculate the BDeu score using the local_score method
        bdeu_local_score = bdeu_score.local_score(ticker, combo)
        #total_score += abs(bdeu_local_score)  # Take the absolute value of the BDeu score
        if bdeu_local_score > result:
            result = bdeu_local_score
            combi = combo
    
    if combi is not None:
        print("Best combination according to BDeu Score :", combi)
        print("Best Score according to BDeu Score : " , result)
    else:
        print("No best combination found.")

    return combi, result

In [ ]:
def create_k2score_df(data, final,ticker):
    k2_combination, k2_score_result = k2_score(data, final,ticker)
    # Extract tickers from data
    all_tickers = [ticker[0] for ticker in data]

    # Create a DataFrame with all tickers
    score_df = pd.DataFrame({'Ticker': all_tickers})

    # Assign scores for K2 (1 for members of the best combination, 0 for others)
    if k2_combination is not None:
        score_df['Parents(K2)'f'{ticker}'] = score_df['Ticker'].apply(lambda x: 1 if x in k2_combination else 0)

    # No need to assign scores for BDeu, so we leave it as is

    return score_df


In [ ]:
start_time = time.time()

# [13 - tickers]
data = [['GOOGL'], ['MSFT'], ['ADBE'], ['AMZN'], ['BA'], ['BRK-B'], ['CSCO'], ['HD'], ['HDFCBANK.NS'], ['IBM'], 
        ['INFY.NS'], ['INTC'], ['JNJ']]

# Assuming 'final' is your Bayesian network model
# You may need to replace this with your actual Bayesian network model

# Create an empty dataframe to store the results
merged_df = pd.DataFrame()

# Loop through each ticker in data
for ticker in data:
    ticker_name = ticker[0]
    
    # Remove the current ticker from data
    data_without_current_ticker = [t for t in data if t != ticker]
    
    # Create score_df excluding the current ticker
    score_df = create_k2score_df(data_without_current_ticker, final, ticker_name)
    print(score_df)
    # Append the current score_df to the merged_df
    merged_df = pd.concat([merged_df, score_df], ignore_index=True)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Best combination according to K2 Score : ['MSFT', 'BRK-B']
Best Score according to K2 Score :  -78.20456108648983
Elapsed time: 4.494576692581177 seconds


In [ ]:
score_df

,Ticker,Parents(K2)AAPL
0,GOOGL,0
1,MSFT,1
2,ADBE,0
3,AMZN,0
4,BA,0
5,BRK-B,1
6,CSCO,0
7,HD,0
